<center>
    <h1>CMSC320 Final Project</h1>
    Valentyn Bolotin, Jacob Barats, Evan May
</center>
<hr>

<center>
    <h3>Introduction</h3>
</center>

Since its creation, alcohol has been a staple in human society. Every country at some point in history has drank alochol and today most countries still do. The difference between then and now is the change in restrictions to alochol. While few countries like Kuwait have <a href="https://www.latimes.com/world/middleeast/la-fg-kuwait-islamists-alcohol-20150105-story.html#:~:text=Kuwait%20and%20Saudi%20Arabia%20are,people%20convicted%20of%20drinking%20alcohol">absolute bans on alcohol</a>, many have regulations in place to limit the amount of alcohol consumption for their citizens. The exact reasoning for these restirctions change depending on the country but the general thought process is that when individuals consume less alcohol, they are better, more productive citizens. 

We want to challenge this reasoning by determining if a countries alochol consumption impacts a countries gross domestic product (GDP). While a country's GDP is not a perfect relection on the productivity of a country, there are enough coorelations between the two that analysis can be conducted.

<hr>

<center>
    <h3>Data collection/curation</h3>
</center>


To determine if alcohol consumption impacts a country's GDP, we need to wrangle data for a country's alcohol consumption and GDP. The problem was that many of these datasets were incomplete or over a small time frame. Our solution was wrangling data from several sources and filling out as much missing information as possible. We have listed all the dataset that we use in this tutorial below.  

* Our World in Data
    * <a href = "https://ourworldindata.org/alcohol-consumption">Alochol Consumption per Capita</a>
    * <a href = "https://ourworldindata.org/grapher/gdp-world-regions-stacked-area?tab=table&time=earliest..1991&country=GRC~DEU~GHA~GTM~GNB~GIN~GMB~GAB~GNQ~GEO">Gross Domestic Product (GDP) based on 2011 International Dollar ($)</a>
    * <a href = "https://ourworldindata.org/grapher/alcohol-consumption-vs-gdp-per-capita">Alochol Consumption per Capita vs GDP per Capita based on Purchasing Power Partiry (PPP) </a>
<br> 
* University of Groningen, Maddison Project Database 2018
    * <a href = "https://www.rug.nl/ggdc/historicaldevelopment/maddison/releases/maddison-project-database-2018">GDP per capita based on 2011 US Dollar with and without multiple benchmarks, Total Population </a>
<br> 
* The World Bank, International Comparison Program (ICP)
    * <a href = "https://ourworldindata.org/alcohol-consumption">Purchasing Power Partiry (PPP)</a>

<hr>

<center>
    <h3>Parsing and Data management</h3>
</center>
To complete the tutorial the following imports are required: Pandas, Numpy
<br>
The warnings import is just to avoid unncessary output and it will still run without it.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pandas.core.arrays import integer
from pandas.core.arrays.numeric import T
import numpy as np

<h4>Finding the Best Range</h4>

As stated previously, the data we aquired had large amounts of missing data. Many countries did not recorded data for various reasons meaning the datasets contain sections of time where there is little to no meaningful data. If we were to take all of the data and preform analysis on it, it would be be too skewed by the missing data to produce any meaningful results. Our decison was to determine the range of years that had the most data.

First we added the dataset from the Maddison Project Database. 

In [2]:
raw_data = pd.read_stata('Data/mpd2018.dta')
raw_df = pd.DataFrame(raw_data)
raw_df = raw_df.rename(columns= {"countrycode" : "code", "pop":"population",
                          "rgdpnapc":"gdp_pc_2011", "cgdppc":"gdp_pc_2011_mb"})
combined_df = raw_df
combined_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,i_cig,i_bm
0,AFG,Afghanistan,1820.0,NaN,NaN,3280.0,NaN,NaN
1,AFG,Afghanistan,1870.0,NaN,NaN,4207.0,NaN,NaN
2,AFG,Afghanistan,1913.0,NaN,NaN,5730.0,NaN,NaN
3,AFG,Afghanistan,1950.0,2392.0,2392.0,8150.0,Extrapolated,NaN
4,AFG,Afghanistan,1951.0,2422.0,2422.0,8284.0,Extrapolated,NaN
...,...,...,...,...,...,...,...,...
19868,ZWE,Zimbabwe,2012.0,1623.0,1604.0,12620.0,Extrapolated,NaN
19869,ZWE,Zimbabwe,2013.0,1801.0,1604.0,13183.0,Extrapolated,NaN
19870,ZWE,Zimbabwe,2014.0,1797.0,1594.0,13772.0,Extrapolated,NaN
19871,ZWE,Zimbabwe,2015.0,1759.0,1560.0,14230.0,Extrapolated,NaN


Once the data was in a dataframe, we first calculated the year ranges that have the highest amount of  (UNSURE OF HOW THIS WORKS)

We also removed any singular data points as they won't be useful in our analysis. 

In [5]:
codes_arr = []

for code in pd.unique(combined_df.code):
  if isinstance(code, str):
    codes_arr.append(code)
combined_df = combined_df[combined_df.code.isin(codes_arr)]

years_arr = np.sort(pd.unique(combined_df.year).reshape(-1))

years_num = {}

for year in years_arr:
  num_entries = len(combined_df[combined_df.year == year].index)
  years_num[year] = num_entries

for i in range(0, len(years_arr)):
  year = years_arr[i]
  prev_year = year
  if i > 0:
    prev_year = years_arr[i-1]
  delta_t = year - prev_year
  delta_v = 0
  delta_vt = 0
  if delta_t > 0:
    delta_v = (years_num[year] - years_num[prev_year][0])
    delta_vt = float(delta_v)/float(delta_t)
  years_num[year] = [years_num[year], years_num[year] - len(codes_arr), delta_t,
                     delta_v, delta_vt]

high_delta_indicies = [] ##[index, delta_abs, series, time_delta, %elts ser, num_entry]

##we are going to assume the data trends up, so lowest offset from max countries
##is the metric we are going to use for large delta jumps, a change in ~20% of 
##data points should do the trick

start_year = years_arr[0]
high_delta_indicies.append([0, np.abs(years_num[start_year][4]), 0, 0, 
          1.0 + float(years_num[start_year][1])/float(len(codes_arr)),
          1])
hd_index = 0

for i in range(1, len(years_arr)):
  year = years_arr[i]
  curr_dvt = np.abs(years_num[year][4])
  curr_dt = years_num[year][2]
  curr_dv = years_num[year][3]
  if(curr_dvt > 0.2 * float(years_num[year][0])):
    hd_index = hd_index + 1
    high_delta_indicies.append([i, curr_dvt, 0, curr_dt,
      1.0 + float(years_num[year][1])/float(len(codes_arr)), 1])
  else:
    tmp = high_delta_indicies[hd_index]
    tmp[2] = tmp[2] + curr_dv
    tmp[3] = tmp[3] + curr_dt
    tmp[4] = tmp[4] + 1.0 + float(years_num[year][1])/float(len(codes_arr))
    tmp[5] = tmp[5] + 1
    high_delta_indicies[hd_index] = tmp


#lets remove any singular data points, theyre not useful

tmp = []
for elt in high_delta_indicies:
  if elt[5] > 1:
    tmp.append(elt)

high_delta_indicies = tmp

#our output
for elt in high_delta_indicies:
  to_print = "Index: {}, year: {}, absDvt: {}, dv_ser: {}, dt: {}, adj_percent: {}"
  to_print = to_print.format(elt[0], years_arr[elt[0]], elt[1], elt[2], elt[3],
                             elt[4]/elt[3])
  print(to_print)

Index: 0, year: 1.0, absDvt: 0, dv_ser: -43, dt: 1279.0, adj_percent: 0.00048577152083497164
Index: 5, year: 1281.0, absDvt: 3.0, dv_ser: 0, dt: 19.0, adj_percent: 0.00591715976331364
Index: 25, year: 1301.0, absDvt: 4.0, dv_ser: 0, dt: 47.0, adj_percent: 0.011834319526627278
Index: 73, year: 1349.0, absDvt: 5.0, dv_ser: 0, dt: 51.0, adj_percent: 0.011834319526627278
Index: 125, year: 1401.0, absDvt: 6.0, dv_ser: 0, dt: 8.0, adj_percent: 0.011834319526627265
Index: 133, year: 1409.0, absDvt: 1.0, dv_ser: 0, dt: 41.0, adj_percent: 0.0177514792899407
Index: 175, year: 1451.0, absDvt: 4.0, dv_ser: 0, dt: 49.0, adj_percent: 0.0177514792899407
Index: 225, year: 1501.0, absDvt: 45.0, dv_ser: 0, dt: 29.0, adj_percent: 0.0177514792899407
Index: 254, year: 1530.0, absDvt: 1.0, dv_ser: 0, dt: 20.0, adj_percent: 0.023668639053254552
Index: 275, year: 1551.0, absDvt: 4.0, dv_ser: 0, dt: 19.0, adj_percent: 0.023668639053254552
Index: 295, year: 1571.0, absDvt: 5.0, dv_ser: 0, dt: 24.0, adj_percent:

Based on the analysis done we can now score each of the ranges and find the most suitable range to conduct our analysis on.

Using the information collected, we now can calculate the scores for each of the year ranges. 

In [7]:
##lets score all of our values
hd_scores = []

for elt in high_delta_indicies:
  score = 0
  ## % covered
  score += elt[4]/elt[3]
  ## ~time step
  time_step = elt[3]/elt[5]
  time_step_score = np.exp(-(time_step - 1))
  score += time_step_score
  ## total entries
  score += elt[5]/len(years_arr)
  ## delta trend
  dt_score = float(elt[2])/float(len(codes_arr))
  dt_score = dt_score * elt[4]/elt[3]
  dt_score = dt_score/2.0 + 0.5
  score += dt_score
  hd_scores.append(score)

max_index = np.argmax(hd_scores)

print("Best index: {}, score: {}, start_year: {}, end_year: {}".format(
  max_index, hd_scores[max_index], years_arr[high_delta_indicies[max_index][0]],
  years_arr[high_delta_indicies[max_index][0]] + high_delta_indicies[max_index][3]
  - years_num[years_arr[high_delta_indicies[max_index][0]]][2]
))

Best index: 27, score: 2.5904183535762484, start_year: 1950.0, end_year: 2016.0


For our tutorial we will use data from 1950 to 2016. This will make it easier to do analysis on as it has the least likelihood of having missing data. We can now remove the unnessesary rows from our dataframe, which is around 8000 rows. 

In [8]:
start_year = years_arr[high_delta_indicies[max_index][0]]
end_year = start_year + high_delta_indicies[max_index][3]

combined_df = combined_df[combined_df.year >= start_year]
combined_df = combined_df[combined_df.year <= end_year]

combined_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,i_cig,i_bm
3,AFG,Afghanistan,1950.0,2392.0,2392.0,8150.0,Extrapolated,NaN
4,AFG,Afghanistan,1951.0,2422.0,2422.0,8284.0,Extrapolated,NaN
5,AFG,Afghanistan,1952.0,2462.0,2462.0,8425.0,Extrapolated,NaN
6,AFG,Afghanistan,1953.0,2568.0,2568.0,8573.0,Extrapolated,NaN
7,AFG,Afghanistan,1954.0,2576.0,2576.0,8728.0,Extrapolated,NaN
...,...,...,...,...,...,...,...,...
19868,ZWE,Zimbabwe,2012.0,1623.0,1604.0,12620.0,Extrapolated,NaN
19869,ZWE,Zimbabwe,2013.0,1801.0,1604.0,13183.0,Extrapolated,NaN
19870,ZWE,Zimbabwe,2014.0,1797.0,1594.0,13772.0,Extrapolated,NaN
19871,ZWE,Zimbabwe,2015.0,1759.0,1560.0,14230.0,Extrapolated,NaN


<h4>Finding the missing values</h4>
While the range of years we have found has the least amount of missing entries, there is still a relative small number. For our purposes in this tutorial, we want to make sure that we have as much GDP data as possible. To fill in this data we will extract the data from the dataset from Our World in Data. 

In [10]:
cpc_df = pd.read_csv('Data/alcohol-consumption-vs-gdp-per-capita.csv')
cpc_df = pd.DataFrame(cpc_df)

cpc_df = cpc_df.drop(labels = "Continent", axis = 1)
cpc_df = cpc_df.rename(columns= {"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)" : "consumption_for_person",
                                 "GDP per capita, PPP (constant 2017 international $)" : "gdp_pc_2017", "Population (historical estimates)": "population_cpc",
                                 "Entity" : "country", "Code":"code", "Year":"year"})
cpc_df = cpc_df[cpc_df.code.isin(codes_arr)]
cpc_df["total_liters_consumed"] = cpc_df.consumption_for_person * cpc_df.population_cpc
cpc_df = cpc_df[cpc_df.year > start_year]
future_cpc_df = cpc_df[cpc_df.year > end_year]

cpc_df = cpc_df[cpc_df.year <= end_year]
cpc_df.gdp_pc_2017 = cpc_df.gdp_pc_2017
cpc_df

,country,code,year,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed
1,Afghanistan,AFG,2010,0.21,1957.029070,29185511.0,6128957.31
2,Afghanistan,AFG,2015,0.21,2068.265904,34413603.0,7226856.63
4,Afghanistan,AFG,2002,NaN,1189.784668,22600774.0,NaN
5,Afghanistan,AFG,2003,NaN,1235.810063,23680871.0,NaN
6,Afghanistan,AFG,2004,NaN,1200.278013,24726689.0,NaN
...,...,...,...,...,...,...,...
57077,Zimbabwe,ZWE,1985,NaN,NaN,8877489.0,NaN
57078,Zimbabwe,ZWE,1986,NaN,NaN,9200150.0,NaN
57079,Zimbabwe,ZWE,1987,NaN,NaN,9527202.0,NaN
57080,Zimbabwe,ZWE,1988,NaN,NaN,9849129.0,NaN


We can search for missing entries in the main dataframe and attempt to fill it with the data in the new dataframe. The dataset from Our World in Data has data on total alcohol consumption per capita so we can add that to our main dataframe. 

In [11]:
hit_years_arr = np.sort(pd.unique(combined_df.year).reshape(-1))

temp_comb = combined_df

cpc_merge = cpc_df.drop(labels=["gdp_pc_2017","consumption_for_person", "population_cpc", "total_liters_consumed"],
                             axis = 1)

for code in codes_arr:
  cpc_code = cpc_merge[cpc_merge.code == code]
  combined_code = combined_df[combined_df.code == code]
  missing_years = np.array(hit_years_arr)
  sub_years = np.array(combined_code.year.to_numpy())
  missing_years = np.setdiff1d(missing_years, sub_years)
  
  if len(missing_years > 0):
    cpc_fill = cpc_code[cpc_code.year.isin(missing_years)]
    cpc_fill.index += len(temp_comb.index)
    temp_comb = pd.concat([temp_comb,cpc_fill])

cpc_merge = cpc_df.drop(labels = ["country"], axis = 1)
temp_comb = pd.merge(how = "left", left = temp_comb, right = cpc_merge, on=["code","year"])
temp_comb["i_cig"] = temp_comb["i_cig"].astype(str)
temp_comb["i_bm"] = temp_comb["i_bm"].astype(str)
temp_comb.i_bm[temp_comb["i_bm"] == "nan"] = pd.NA
temp_comb.i_bm[temp_comb["i_cig"] == "nan"] = pd.NA
temp_comb = temp_comb.fillna(-1)

combined_df = temp_comb
combined_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,i_cig,i_bm,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed
0,AFG,Afghanistan,1950.0,2392.0,2392.0,8150.0,Extrapolated,-1,-1.00,-1.000000,-1.0,-1.00
1,AFG,Afghanistan,1951.0,2422.0,2422.0,8284.0,Extrapolated,-1,-1.00,-1.000000,7840151.0,-1.00
2,AFG,Afghanistan,1952.0,2462.0,2462.0,8425.0,Extrapolated,-1,-1.00,-1.000000,7935996.0,-1.00
3,AFG,Afghanistan,1953.0,2568.0,2568.0,8573.0,Extrapolated,-1,-1.00,-1.000000,8039684.0,-1.00
4,AFG,Afghanistan,1954.0,2576.0,2576.0,8728.0,Extrapolated,-1,-1.00,-1.000000,8151316.0,-1.00
...,...,...,...,...,...,...,...,...,...,...,...,...
11318,ZWE,Zimbabwe,2012.0,1623.0,1604.0,12620.0,Extrapolated,-1,-1.00,3170.726522,13115149.0,-1.00
11319,ZWE,Zimbabwe,2013.0,1801.0,1604.0,13183.0,Extrapolated,-1,-1.00,3176.829330,13350378.0,-1.00
11320,ZWE,Zimbabwe,2014.0,1797.0,1594.0,13772.0,Extrapolated,-1,-1.00,3195.767970,13586710.0,-1.00
11321,ZWE,Zimbabwe,2015.0,1759.0,1560.0,14230.0,Extrapolated,-1,4.92,3198.982129,13814642.0,67968038.64


We only want data that has a low amount of missing entries. Data that is missing all of its population data isn't helpful so we remove them.

In [12]:
##first going to see if there is a year after which population is recorded for 
##any country

missing_codes = {}
for code in codes_arr:
  pop = combined_df[combined_df.code == code]
  pop = pop[((pop.population < 0) | (pop.population_cpc < 0))]
  ##print(pop[pop.population_cpc <= 0])
  pop_arr = []
  for i, row in pop.iterrows():
    pop_arr.append(row.year)
  missing_codes[code] = pop_arr

missing_codes

def code_comp (a):
  return -len(missing_codes[a])

def code_comp (a):
      return -len(missing_codes[a])

sorted_codes = sorted(codes_arr, key=code_comp)

for code in sorted_codes:
  print(code, ": ", len(missing_codes[code]))

CSK :  67
SUN :  67
YUG :  67
SRB :  2
AFG :  1
AGO :  1
ALB :  1
ARE :  1
ARG :  1
ARM :  1
AUS :  1
AUT :  1
AZE :  1
BDI :  1
BEL :  1
BEN :  1
BFA :  1
BGD :  1
BGR :  1
BHR :  1
BIH :  1
BLR :  1
BOL :  1
BRA :  1
BRB :  1
BWA :  1
CAF :  1
CAN :  1
CHE :  1
CHL :  1
CHN :  1
CIV :  1
CMR :  1
COD :  1
COG :  1
COL :  1
COM :  1
CPV :  1
CRI :  1
CUB :  1
CYP :  1
CZE :  1
DEU :  1
DJI :  1
DMA :  1
DNK :  1
DOM :  1
DZA :  1
ECU :  1
EGY :  1
ESP :  1
EST :  1
ETH :  1
FIN :  1
FRA :  1
GAB :  1
GBR :  1
GEO :  1
GHA :  1
GIN :  1
GMB :  1
GNB :  1
GNQ :  1
GRC :  1
GTM :  1
HKG :  1
HND :  1
HRV :  1
HTI :  1
HUN :  1
IDN :  1
IND :  1
IRL :  1
IRN :  1
IRQ :  1
ISL :  1
ISR :  1
ITA :  1
JAM :  1
JOR :  1
JPN :  1
KAZ :  1
KEN :  1
KGZ :  1
KHM :  1
KOR :  1
KWT :  1
LAO :  1
LBN :  1
LBR :  1
LBY :  1
LCA :  1
LKA :  1
LSO :  1
LTU :  1
LUX :  1
LVA :  1
MAR :  1
MDA :  1
MDG :  1
MEX :  1
MKD :  1
MLI :  1
MLT :  1
MMR :  1
MNE :  1
MNG :  1
MOZ :  1
MRT :  1
MUS :  1
MWI :  

In [13]:
rem_codes = []
for code in sorted_codes:
  if len(missing_codes[code]) == end_year - start_year:
    rem_codes.append(code)

combined_df = combined_df[~combined_df.code.isin(rem_codes)]
hit_codes = pd.unique(combined_df.code).reshape(-1)
combined_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,i_cig,i_bm,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed
0,AFG,Afghanistan,1950.0,2392.0,2392.0,8150.0,Extrapolated,-1,-1.00,-1.000000,-1.0,-1.00
1,AFG,Afghanistan,1951.0,2422.0,2422.0,8284.0,Extrapolated,-1,-1.00,-1.000000,7840151.0,-1.00
2,AFG,Afghanistan,1952.0,2462.0,2462.0,8425.0,Extrapolated,-1,-1.00,-1.000000,7935996.0,-1.00
3,AFG,Afghanistan,1953.0,2568.0,2568.0,8573.0,Extrapolated,-1,-1.00,-1.000000,8039684.0,-1.00
4,AFG,Afghanistan,1954.0,2576.0,2576.0,8728.0,Extrapolated,-1,-1.00,-1.000000,8151316.0,-1.00
...,...,...,...,...,...,...,...,...,...,...,...,...
11318,ZWE,Zimbabwe,2012.0,1623.0,1604.0,12620.0,Extrapolated,-1,-1.00,3170.726522,13115149.0,-1.00
11319,ZWE,Zimbabwe,2013.0,1801.0,1604.0,13183.0,Extrapolated,-1,-1.00,3176.829330,13350378.0,-1.00
11320,ZWE,Zimbabwe,2014.0,1797.0,1594.0,13772.0,Extrapolated,-1,-1.00,3195.767970,13586710.0,-1.00
11321,ZWE,Zimbabwe,2015.0,1759.0,1560.0,14230.0,Extrapolated,-1,4.92,3198.982129,13814642.0,67968038.64


<h4>Additional tidying</h4>
We want to make sure that all of the data is in the same units. We change our population's scale to be one to one and make both our GDP columns be in terms of GDP per capita. This avoids the problem of unit conversion for analysis later. The missing entries were changed to -1 and the population and years were changed to integers instead of strings.

In [14]:
st_df = combined_df

st_df["population"] = st_df["population"] * 1000
st_df.population[st_df.population < 0] = -1

st_df["gdp_2011"] = st_df["gdp_pc_2011"] * st_df["population"]
st_df.gdp_2011[st_df.gdp_2011 < 0] = -1

st_df["gdp_2011_mb"] = st_df["gdp_pc_2011_mb"] * st_df["population"]
st_df.gdp_2011_mb[st_df.gdp_2011_mb < 0] = -1

st_df["gdp_2017"] = st_df["gdp_pc_2017"] * st_df["population_cpc"]
st_df.gdp_2017[st_df.gdp_2017 < 0] = -1

st_df["population"] = st_df["population"].astype(int)
st_df["population_cpc"] = st_df["population_cpc"].astype(int)
st_df["year"] = st_df["year"].astype(int)
st_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,i_cig,i_bm,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,gdp_2011,gdp_2011_mb,gdp_2017
0,AFG,Afghanistan,1950,2392.0,2392.0,8150000,Extrapolated,-1,-1.00,-1.000000,-1,-1.00,1.949480e+10,1.949480e+10,1.000000e+00
1,AFG,Afghanistan,1951,2422.0,2422.0,8284000,Extrapolated,-1,-1.00,-1.000000,7840151,-1.00,2.006385e+10,2.006385e+10,-1.000000e+00
2,AFG,Afghanistan,1952,2462.0,2462.0,8425000,Extrapolated,-1,-1.00,-1.000000,7935996,-1.00,2.074235e+10,2.074235e+10,-1.000000e+00
3,AFG,Afghanistan,1953,2568.0,2568.0,8573000,Extrapolated,-1,-1.00,-1.000000,8039684,-1.00,2.201546e+10,2.201546e+10,-1.000000e+00
4,AFG,Afghanistan,1954,2576.0,2576.0,8728000,Extrapolated,-1,-1.00,-1.000000,8151316,-1.00,2.248333e+10,2.248333e+10,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11318,ZWE,Zimbabwe,2012,1623.0,1604.0,12620000,Extrapolated,-1,-1.00,3170.726522,13115149,-1.00,2.024248e+10,2.048226e+10,4.158455e+10
11319,ZWE,Zimbabwe,2013,1801.0,1604.0,13183000,Extrapolated,-1,-1.00,3176.829330,13350378,-1.00,2.114553e+10,2.374258e+10,4.241187e+10
11320,ZWE,Zimbabwe,2014,1797.0,1594.0,13772000,Extrapolated,-1,-1.00,3195.767970,13586710,-1.00,2.195257e+10,2.474828e+10,4.341997e+10
11321,ZWE,Zimbabwe,2015,1759.0,1560.0,14230000,Extrapolated,-1,4.92,3198.982129,13814642,67968038.64,2.219880e+10,2.503057e+10,4.419279e+10


The 2017 GDP that was added to our dataset is in terms of the constant 2017 internation dollar which is different than the 2011 GDP and the 2011 MB GDP. This causes a problem when the they are compared to one another in an analytical setting. Conversion factors are calculated and put in the dataset. 

In [35]:
##Fill missing economic datapoints
##First find conversion factors

y1_df = combined_df[combined_df.year == 2011]
y2_df = combined_df[combined_df.year == 2016]
y1_vals = {"gdp_2011":0, "gdp_2011_mb":0, "gdp_2017":0}
y2_vals = {"gdp_2011":0, "gdp_2011_mb":0, "gdp_2017":0}
for code in hit_codes:
  c_df = y1_df[combined_df.code == code]
  for v in c_df["gdp_2011"]:
    y1_vals["gdp_2011"] = y1_vals["gdp_2011"] + v
  for v in c_df["gdp_2011_mb"]:
    y1_vals["gdp_2011_mb"] = y1_vals["gdp_2011_mb"] + v
  for v in c_df["gdp_2017"]:
    y1_vals["gdp_2017"] = y1_vals["gdp_2017"] + v
  c_df = y2_df[combined_df.code == code]
  for v in c_df["gdp_2011"]:
    y2_vals["gdp_2011"] = y2_vals["gdp_2011"] + v
  for v in c_df["gdp_2011_mb"]:
    y2_vals["gdp_2011_mb"] = y2_vals["gdp_2011_mb"] + v
  for v in c_df["gdp_2017"]:
    y2_vals["gdp_2017"] = y2_vals["gdp_2017"] + v

y1_vals["fac1"] = y1_vals["gdp_2011"] / y1_vals["gdp_2017"]
y1_vals["fac2"] = y1_vals["gdp_2011_mb"] / y1_vals["gdp_2017"]
y1_vals["fac2011"] = y1_vals["gdp_2011_mb"] / y1_vals["gdp_2011"]

y2_vals["fac1"] = y2_vals["gdp_2011"] / y2_vals["gdp_2017"]
y2_vals["fac2"] = y2_vals["gdp_2011_mb"] / y2_vals["gdp_2017"]
y2_vals["fac2011"] = y2_vals["gdp_2011_mb"] / y2_vals["gdp_2011"]

print(y1_vals)
print(y2_vals)

combined_df[combined_df.code == "USA"]

{'gdp_2011': 93952331058496.0, 'gdp_2011_mb': 93952331058496.0, 'gdp_2017': 98171539732185.94, 'fac1': 0.957022079054683, 'fac2': 0.957022079054683, 'fac2011': 1.0}
{'gdp_2011': 107295416080637.0, 'gdp_2011_mb': 106491490024125.0, 'gdp_2017': 115602680525130.58, 'fac1': 0.9281395171222896, 'fac2': 0.92118530072471, 'fac2011': 0.9925073587868114}


,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,i_cig,i_bm,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,gdp_2011,gdp_2011_mb,gdp_2017
10720,USA,United States,1950,15241.0,15241.0,152271008,Interpolated,-1,-1.00,-1.000000,-1,-1.000000e+00,2.320762e+12,2.320762e+12,1.000000e+00
10721,USA,United States,1951,16126.0,16126.0,154878000,Interpolated,-1,-1.00,-1.000000,160872264,-1.000000e+00,2.497563e+12,2.497563e+12,-1.000000e+00
10722,USA,United States,1952,16443.0,16443.0,157552992,Interpolated,-1,-1.00,-1.000000,163266026,-1.000000e+00,2.590644e+12,2.590644e+12,-1.000000e+00
10723,USA,United States,1953,16917.0,16917.0,160184000,Interpolated,-1,-1.00,-1.000000,165909996,-1.000000e+00,2.709833e+12,2.709833e+12,-1.000000e+00
10724,USA,United States,1954,16513.0,16513.0,163026000,Interpolated,-1,-1.00,-1.000000,168736390,-1.000000e+00,2.692048e+12,2.692048e+12,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10782,USA,United States,2012,50394.0,50394.0,314743008,Extrapolated,-1,-1.00,55581.413844,314043885,-1.000000e+00,1.586116e+13,1.586116e+13,1.745500e+13
10783,USA,United States,2013,50863.0,50863.0,317072992,Extrapolated,-1,-1.00,56214.428677,316400539,-1.000000e+00,1.612728e+13,1.612728e+13,1.778628e+13
10784,USA,United States,2014,51664.0,51664.0,319558016,Extrapolated,-1,-1.00,57213.267841,318673422,-1.000000e+00,1.650965e+13,1.650965e+13,1.823235e+13
10785,USA,United States,2015,52591.0,52591.0,322073984,Extrapolated,-1,9.82,58540.296093,320878312,3.151025e+09,1.693819e+13,1.693819e+13,1.878431e+13


<h4>Created the world dataset</h4>
We now have data for every country and year. We want to compare the data from that dataset to the sum of all the data. We create a world dataset to accomplish this.

In [15]:
world_df = pd.DataFrame(columns= combined_df.columns).drop(labels = ["i_cig", "i_bm"], axis = 1)
world_miss_df = pd.DataFrame(columns= world_df.columns).drop(labels = ["country", "code"], axis = 1)

common_attrs = ["population",
                "population_cpc",
                "consumption_for_person",
                "gdp_pc_2011_mb",
                "gdp_pc_2011",
                "gdp_pc_2017",
                "total_liters_consumed",
                "gdp_2011_mb",
                "gdp_2011",
                "gdp_2017"]

frac_attrs =   ["gdp_pc_2011_mb",
                "gdp_pc_2011",
                "gdp_pc_2017",
                "consumption_for_person"]

map_attrs =    {"gdp_pc_2011_mb" : "gdp_2011_mb",
                "gdp_pc_2011" : "gdp_2011",
                "gdp_pc_2017": "gdp_2017",
                "consumption_for_person" : "total_liters_consumed"}

for year in hit_years_arr:
  year_df = combined_df[combined_df.year == year]
  ser = pd.Series()
  ser["country"] = "WORLD"
  ser["code"] = "WORLD"
  ser["year"] = year

  ser_miss = pd.Series()
  ser_miss["year"] = year
  
  for attr in common_attrs:
    ser[attr] = 0
    ser_miss[attr] = []

  for i, row in year_df.iterrows():
    for attr in common_attrs:
      if row[attr] > 0:
        ser[attr] = row[attr] + ser[attr]
      else:
        ser_miss[attr].append(row["code"])

  for attr in common_attrs:
    if ser[attr] == 0:
      ser[attr] = -1

      
  # FIX FRAC ATTRS &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
  for attr in frac_attrs:
    if ser[attr] > 0:
      ser[attr] = ser[map_attrs[attr]] / ser["population"]
  world_df.loc[len(world_df.index)] = ser
  world_miss_df.loc[len(world_miss_df.index)] = ser_miss
  ##world_df = pd.concat([world_df,pd.Series(vs)], axis = 0, ignore_index = True)
  ##world_miss_df = pd.concat([world_miss_df,ser_miss], axis = 0)
world_df

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,gdp_2011,gdp_2011_mb,gdp_2017
0,WORLD,WORLD,1950.0,2840.316642,3193.007878,2514160984,-1,-1,-1,-1,8027735828641.0,7141013283921.0,166.0
1,WORLD,WORLD,1951.0,2985.611376,3355.141447,2557931000,-1,-1,2572198645,-1,8582220316169.0,7636987893105.0,-1
2,WORLD,WORLD,1952.0,3067.791124,3456.668876,2610876008,-1,-1,2618839562,-1,9024933836048.0,8009622243888.0,-1
3,WORLD,WORLD,1953.0,3191.232846,3591.33825,2658728016,-1,-1,2665387479,-1,9548391621048.0,8484620172448.0,-1
4,WORLD,WORLD,1954.0,3204.739537,3626.651794,2709558016,-1,-1,2712412149,-1,9826623440912.0,8683427700912.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,WORLD,WORLD,2012.0,13815.576851,13817.003207,6992922992,-1,14492.744916,7079608754,-1,96621239410112.0,96611265008512.0,101346649138136.046875
63,WORLD,WORLD,2013.0,14030.044051,14084.76023,7072213016,-1,14806.919497,7163434702,-1,99610424623616.0,99223460153152.0,104717688794875.0625
64,WORLD,WORLD,2014.0,14261.271531,14366.788468,7152269064,-1,15140.486309,7247241265,-1,102755136711424.0,102000451182720.0,108288831838648.9375
65,WORLD,WORLD,2015.0,14493.906019,14601.919766,7231374064,6.175409,15485.600274,7330861223,44656690316.553009,105591943876928.0,104810856072960.0,111982168185818.25


Using this data we can but the fraction of the GDP, alcohol consumed, and population that a country represents for a given year. This is placed in the main dataframe.

In [16]:
comb_df = pd.concat([combined_df,world_df], ignore_index = True)
comb_df = comb_df.drop(labels = ["i_cig","i_bm"], axis = 1)

comb_df["frac_gdp_2011"] = -1
comb_df["frac_gdp_2011_mb"] = -1
comb_df["frac_gdp_2017"] = -1
comb_df["frac_liters_consumed"] = -1
comb_df["frac_population"] = -1
comb_df["frac_population_cpc"] = -1

for year in hit_years_arr:
  total = world_df[world_df.year == year].gdp_2011.to_numpy()[0]
  comb_df.frac_gdp_2011[comb_df.year == year] = comb_df.gdp_2011 / total

  total = world_df[world_df.year == year].gdp_2011_mb.to_numpy()[0]
  comb_df.frac_gdp_2011_mb[comb_df.year == year] = comb_df.gdp_2011_mb / total

  total = world_df[world_df.year == year].gdp_2017.to_numpy()[0]
  comb_df.frac_gdp_2017[comb_df.year == year] = comb_df.gdp_2017 / total

  total = world_df[world_df.year == year].total_liters_consumed.to_numpy()[0]
  comb_df.frac_liters_consumed[comb_df.year == year] = comb_df.total_liters_consumed / total

  total = world_df[world_df.year == year].population.to_numpy()[0]
  comb_df.frac_population[comb_df.year == year] = comb_df.population / total

  total = world_df[world_df.year == year].population_cpc.to_numpy()[0]
  comb_df.frac_population_cpc[comb_df.year == year] = comb_df.population_cpc / total

comb_df.frac_gdp_2011[comb_df["gdp_2011"] == -1] = -1
comb_df.frac_gdp_2011_mb[comb_df["gdp_2011_mb"] == -1] = -1
comb_df.frac_gdp_2017[comb_df["gdp_2017"] == -1] = -1
comb_df.frac_liters_consumed[comb_df["total_liters_consumed"] == -1] = -1
comb_df.frac_population[comb_df["population"] == -1] = -1
comb_df.frac_population_cpc[comb_df["population_cpc"] == -1] = -1

comb_df[comb_df.code == "USA"]

,code,country,year,gdp_pc_2011_mb,gdp_pc_2011,population,consumption_for_person,gdp_pc_2017,population_cpc,total_liters_consumed,gdp_2011,gdp_2011_mb,gdp_2017,frac_gdp_2011,frac_gdp_2011_mb,frac_gdp_2017,frac_liters_consumed,frac_population,frac_population_cpc
10586,USA,United States,1950,15241.0,15241.0,152271008,-1.0,-1.0,-1,-1.0,2320762404864.0,2320762404864.0,1.0,0.289093,0.324991,0.006024,-1,0.060565,-1
10587,USA,United States,1951,16126.0,16126.0,154878000,-1.0,-1.0,160872264,-1.0,2497562542080.0,2497562542080.0,-1.0,0.291016,0.327035,-1,-1,0.060548,0.062543
10588,USA,United States,1952,16443.0,16443.0,157552992,-1.0,-1.0,163266026,-1.0,2590643847168.0,2590643847168.0,-1.0,0.287054,0.323441,-1,-1,0.060345,0.062343
10589,USA,United States,1953,16917.0,16917.0,160184000,-1.0,-1.0,165909996,-1.0,2709832597504.0,2709832597504.0,-1.0,0.2838,0.319382,-1,-1,0.060248,0.062246
10590,USA,United States,1954,16513.0,16513.0,163026000,-1.0,-1.0,168736390,-1.0,2692048224256.0,2692048224256.0,-1.0,0.273955,0.310021,-1,-1,0.060167,0.062209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10648,USA,United States,2012,50394.0,50394.0,314743008,-1.0,55581.413844,314043885,-1.0,15861159034880.0,15861159034880.0,17455003137211.960938,0.164158,0.164175,0.172231,-1,0.045009,0.044359
10649,USA,United States,2013,50863.0,50863.0,317072992,-1.0,56214.428677,316400539,-1.0,16127283429376.0,16127283429376.0,17786275533020.800781,0.161904,0.162535,0.16985,-1,0.044834,0.044169
10650,USA,United States,2014,51664.0,51664.0,319558016,-1.0,57213.267841,318673422,-1.0,16509645619200.0,16509645619200.0,18232347846554.539062,0.16067,0.161859,0.168368,-1,0.044679,0.043972
10651,USA,United States,2015,52591.0,52591.0,322073984,9.82,58540.296093,320878312,3151025023.84,16938193387520.0,16938193387520.0,18784311394301.296875,0.160412,0.161607,0.167744,0.070561,0.044538,0.043771


<h4>Reading the Data</h4>
We downloaded the CSV files from Our World in Data. First we created the created the dataframe for GDP by country and year.

In [26]:
combined_df = pd.read_csv('Data/gdp-world-regions-stacked-area.csv')
combined_df = pd.DataFrame(combined_df)
combined_df.head()


,Entity,Code,Year,GDP,417484-annotations
0,Afghanistan,AFG,1950,9421400064,NaN
1,Afghanistan,AFG,1951,9692279808,NaN
2,Afghanistan,AFG,1952,10017325056,NaN
3,Afghanistan,AFG,1953,10630519808,NaN
4,Afghanistan,AFG,1954,10866360320,NaN


This code is messy and has uncessesary columns. Let's tidy the dataframe so it's organized and clear.

In [27]:
combined_df = combined_df[combined_df.Year != 1]
combined_df = combined_df.drop(labels = "417484-annotations", axis=1)
combined_df = combined_df.rename(columns= {"Entity" : "Country"})
combined_df = combined_df[combined_df.Country != "World"]
combined_df.head()

,Country,Code,Year,GDP
0,Afghanistan,AFG,1950,9421400064
1,Afghanistan,AFG,1951,9692279808
2,Afghanistan,AFG,1952,10017325056
3,Afghanistan,AFG,1953,10630519808
4,Afghanistan,AFG,1954,10866360320



<ol>
    <li>What the data means</li>
</ol>


<ol>
    <li>Explanations of what the columns mean (the important columns that we will use)</li>
    <li>Did we need to parse the data at all?</li>
    <li>Show how we checked if there were any missing values and how we handled them</li>
    <li>Did we need to tidy the data/remove unnessesary data</li>
  </ol>
<hr>

<center>
    <h3>Exploratory data analysis</h3>
</center>
<ol>
    <li>How did we want to display the data</li>
    <li>Code of displaying data</li>
    <li>Graphs/Charts of displayed data</li>
    </ol>
There will be multiple groups of these three sections
<hr>


<center>
    <h3>Hypothesis testing</h3>
</center>
<ol>
    <li>Conduct hypothesis testing</li>
    <li>Code of displaying data</li>
    <li>Show results in multiple formats (graphs, data, etc)</li>
  </ol>
<hr>

<center>
    <h3>Communication of insights attained</h3>
</center>
  <ol>
    <li>Descirbe what this data means (give multiple interpretations)</li>
    <li>Give reasoning as to why the results were like this (any error that could have influenced results?)</li>
    <li>Conclusion</li>
  </ol>
<hr>